# Functions for reading a graph from a file and writing to the file from the graph.

In [128]:
import numpy as np
import pandas as pd
import networkx as nx
import pickle

import matplotlib.pyplot as plt

In [129]:
def graph_from_input(filename):
    '''graph_from_input(str) --> nx.graph, np.ndarray[int], locToIndex
    Returns a graph created by reading the input file, with integer vertex labels
    Returns list of the home indices
    Returns a map from integer to the name associated with that node'''
    with open(filename, 'r') as f:
        G = nx.Graph()
        
        locToIndex = {} # maps location name to its index number
        homes = []
        lines = f.readlines()
        
        numLocations = int(lines[0])
        numTAs = int(lines[1])
        locations = lines[2].split()
    
        i = 0
        assert len(locations) == numLocations, "Number of locations must match specified value"
        for loc in locations:
            G.add_node(i)
            locToIndex[loc] = i
            i += 1
            
        TAhomes = lines[3].split()
        assert len(TAhomes) == numTAs, "Number of TA homes must match specified value"
        for home in TAhomes:
            homes.append(locToIndex[home])
        
        homes.insert(0, locToIndex[lines[4].strip()])
        
        row = 0
        for line in lines[5:]:
            line = line.split()
            for col in range(len(line)):
            
                if line[col] != 'x':  
                    G.add_edge(row, col)
                    weight = float(line[col])
                    G[row][col]['weight'] = weight
        
        indexToLoc = {v: k for k, v in locToIndex.items()}
        return G, homes, indexToLoc

(<networkx.classes.graph.Graph at 0x10b1faac8>,
 [58,
  55,
  71,
  65,
  54,
  67,
  42,
  45,
  84,
  91,
  29,
  11,
  92,
  47,
  3,
  53,
  39,
  76,
  83,
  68,
  26,
  64,
  44,
  57,
  50],
 {0: 'Building0',
  1: 'Building1',
  2: 'Building2',
  3: 'Building3',
  4: 'Building4',
  5: 'Building5',
  6: 'Building6',
  7: 'Building7',
  8: 'Building8',
  9: 'Building9',
  10: 'Building10',
  11: 'Building11',
  12: 'Building12',
  13: 'Building13',
  14: 'Building14',
  15: 'Building15',
  16: 'Building16',
  17: 'Building17',
  18: 'Building18',
  19: 'Building19',
  20: 'Building20',
  21: 'Building21',
  22: 'Building22',
  23: 'Building23',
  24: 'Building24',
  25: 'Building25',
  26: 'Building26',
  27: 'Building27',
  28: 'Building28',
  29: 'Building29',
  30: 'Building30',
  31: 'Building31',
  32: 'Building32',
  33: 'Building33',
  34: 'Building34',
  35: 'Building35',
  36: 'Building36',
  37: 'Building37',
  38: 'Building38',
  39: 'Building39',
  40: 'Building40',
  

In [130]:
def write_input_file(G, homes, filename):
    '''write_graph(nx.graph, str) --> None
    Write a graph G to the file pointed to by filename.
    The file will be written in the input format as specified
    in the project spec.'''
    assert isinstance(G, nx.Graph) , "G must be a graph"
    assert isinstance(homes, np.ndarray) , "homes must be an np array of integers"
    assert isinstance(filename, str) , "filename must be a string"
    
    locations = G.nodes
    #print(locations)
    num_homes = len(homes) - 1
    with open(filename, 'w') as f:
        f.write(str(len(locations)) + "\n")
        f.write(str(num_homes) + "\n")
        
        for g in G.nodes:
            f.write("Building{} ".format(g))
        f.write('\n')
    
        for h in homes[1:]:
            f.write("Building{} ".format(h))
        f.write('\n')
        
        f.write("Building{} \n".format(homes[0])) # Starting location, usually Soda
        
        adjMat = nx.to_numpy_array(G)
        for row in adjMat:
            for elt in row:
                if elt == 0:
                    f.write('x ')
                else:    
                     f.write(str(elt) + ' ')
            f.write('\n')

In [135]:
def write_output_file(path, dropOffs, indexToName, filename):
    '''path is a list of integers that we follow in the car 
    dropOffs is a dictionary (int -> [home, home, ...] ) mapping nodes to the homes of the TAs that get off at that node 
    indexToName is a list of names corresponding to each index
    filename is a string filename that we write to  
    '''
    with open(filename, 'w') as f:
        for step in path:
            f.write(indexToName[step] + " ")
        
        f.write("\n")
        count = 0
        for key in dropOffs:
            if dropOffs[key] != []:
                count += 1
        f.write(str(count) + "\n")
        
        for key in dropOffs:
            if dropOffs[key] != []:
                f.write(indexToName[key] + " ")
                for elt in dropOffs[key]:
                    f.write(indexToName[elt] + " ");
                f.write("\n")

In [ ]:
G = nx.readwrite.gpickle.read_gpickle('./graphstore/tempG.pkl')
with open('./graphstore/homes.pkl', 'rb') as f: homes = pickle.load(f)
    
write_input_file(G, homes, "out.txt")

graph_from_input("out.txt")

with open('./graphstore/where.pkl', 'rb') as f: dropOffs = pickle.load(f)
with open('./graphstore/final_path.pkl', 'rb') as f: path = pickle.load(f)
    
indexToName = {i : "Building{}".format(i) for i in range(100)}
write_output_file(path, dropOffs, indexToName, "output_file.txt")